# **Neural Network for Image Classification**
*   Implementation of a classic neural network for image classification supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a classic neural network for image classification supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.
*   Implementations are based off of TensorFlow's very own example: [TensorFlow Image Claasification](https://www.tensorflow.org/tutorials/keras/classification)

In [1]:
import time
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist

In [2]:
# Set a global TF random seed
tf.random.set_seed(12)

In [3]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 5
# Values are specific to the Fashion MNIST dataset
INPUT_DIM = 28
NUM_CLASSES = 10

In [4]:
def build_and_train(X_train, y_train, precision):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Flatten(input_shape=(INPUT_DIM, INPUT_DIM)),
        layers.Dense(128, activation='relu', dtype=dtype),
        layers.Dense(NUM_CLASSES, dtype=dtype)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [5]:
def build_and_train_mixed(X_train, y_train):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    model = models.Sequential([
        layers.Flatten(input_shape=(INPUT_DIM, INPUT_DIM)),
        layers.Dense(128, activation='relu'),
        layers.Dense(NUM_CLASSES)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [6]:
# Load dataset and split into train and test sets
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Scale values before feeding into neural net
X_train = X_train / 255.0
X_test  = X_test / 255.0

4422102/4422102 [==============================] - 1s 0us/step


In [7]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')
_ = build_and_train_mixed(X_train, y_train)

Epoch 1/5
1875/1875 [==============================] - 6s 2ms/step - loss: 0.4970 - accuracy: 0.8242
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3751 - accuracy: 0.8653
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3367 - accuracy: 0.8771
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3137 - accuracy: 0.8835
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2943 - accuracy: 0.8907
Epoch 1/5
1875/1875 [==============================] - 8s 3ms/step - loss: 0.4953 - accuracy: 0.8260
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3720 - accuracy: 0.8664
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3336 - accuracy: 0.8787
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3120 - accuracy: 0.8846
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2937 - accurac

In [16]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5056 - accuracy: 0.8208
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3790 - accuracy: 0.8631
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3390 - accuracy: 0.8760
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3160 - accuracy: 0.8834
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2949 - accuracy: 0.8914
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.5002 - accuracy: 0.8241
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3752 - accuracy: 0.8651
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3370 - accuracy: 0.8768
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3149 - accuracy: 0.8847
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2957 - accurac

In [9]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4959 - accuracy: 0.8264
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3728 - accuracy: 0.8654
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3354 - accuracy: 0.8775
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3137 - accuracy: 0.8849
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2955 - accuracy: 0.8905
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4933 - accuracy: 0.8261
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3736 - accuracy: 0.8653
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3347 - accuracy: 0.8780
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3126 - accuracy: 0.8845
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2943 - accurac

In [13]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 19.4879 - accuracy: 0.6748
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 7.4060 - accuracy: 0.7695
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 4.8795 - accuracy: 0.7950
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 3.7466 - accuracy: 0.8036
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 3.0589 - accuracy: 0.8114
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 18.1008 - accuracy: 0.6623
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 5.0677 - accuracy: 0.7652
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 3.5673 - accuracy: 0.7891
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.7677 - accuracy: 0.8028
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.2976 - accur

In [18]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(X_train, y_train)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4942 - accuracy: 0.8262
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3740 - accuracy: 0.8650
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3336 - accuracy: 0.8781
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3113 - accuracy: 0.8849
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2923 - accuracy: 0.8921
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4996 - accuracy: 0.8244
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3760 - accuracy: 0.8653
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3364 - accuracy: 0.8774
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3140 - accuracy: 0.8856
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2955 - accurac

In [19]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 35.598839616775514 seconds
Average training time in single precision: 32.36794843673706 seconds
Average training time in half precision: 38.933386182785036 seconds
Average training time in mixed half precision: 35.02050609588623 seconds
-------------
Accuracy with double precision: 0.8727999925613403
Accuracy with single precision: 0.8755999803543091
Accuracy with half precision: 0.7911999821662903
Accuracy with mixed half precision: 0.8723000288009644
